In [2]:
import pyprob
from pyprob.distributions import Normal
import pyprob.diagnostics
import torch
import pandas
import os

def parse_n_infect_from_output(file_name):
    data = pandas.read_csv(file_name, sep='\t', names = ["survey_number", "third_dimension", "measure", "value"])
    n_infect = torch.from_numpy(data['value'][:12].values)
    return n_infect

PATH = '/home/examples/test/'
def forward_before():
    exists = os.path.isfile(PATH + 'output.txt')
    if exists:
        print('Removing output...')
        os.remove(PATH+'output.txt')
    
def forward_after():
    n_infect = parse_n_infect_from_output(PATH + 'output.txt')
    likelihood = Normal(n_infect, 0.01)
    pyprob.observe(likelihood, name='n_infect')

# This will save in /code/notebooks
file_name = 'trace'
address_dict_file_name = 'address_dict'
use_address_base=True
model = pyprob.RemoteModel('ipc://@openmalaria_probprog', before_forward_func=forward_before ,after_forward_func=forward_after, address_dict_file_name=address_dict_file_name)

In [ ]:
trace = next(model._trace_generator())

ppx (Python): zmq.REQ socket connecting to server ipc://@openmalaria_probprog
ppx (Python): This system        : pyprob 0.13.2.dev18
ppx (Python): Connected to system: pyprob_cpp 0.1.9 (master:7290be2)
ppx (Python): Model name         : OpenMalaria probprog
Removing output...


## Generating the Prior

In [2]:
prior = model.prior_traces(num_traces=5)

Time spent  | Time remain.| Progress             | Trace | Traces/sec
ppx (Python): zmq.REQ socket connecting to server ipc://@openmalaria_probprog
ppx (Python): This system        : pyprob 0.13.2.dev18
ppx (Python): Connected to system: pyprob_cpp 0.1.9 (master:7290be2)
ppx (Python): Model name         : OpenMalaria probprog
Removing output...
Removing output...0:13:15 | ########------------ | 2/5 | 0.01       
Removing output...0:13:53 | ############-------- | 3/5 | 0.00       
Removing output...0:10:34 | ################---- | 4/5 | 0.00       
Removing output...0:05:41 | #################### | 5/5 | 0.00       



## Analysing the Prior

In [5]:
# print(len(prior))
# trace=prior._values[0]
prior._values
trace=prior._values[1]
var = trace.variables
# len(var)
# print(var)
count  = 1
for k in trace.variables_dict_address_base:
    print(" Address {2} : {0} \n Value : {1} \n\n".format(k,trace.variables_dict_address_base[k],count))
    count += 1
#pyprob.diagnostics.graph(prior, file_name="plots/mpenny_adapted_17")
#print(prior.save_metadata)

 Address 1 : __A25 
 Value : Variable(name:None, control:True, replace:False, observable:False, observed:False, tagged:False, address:__A25__17, distribution:Normal(mean:tensor([0.]), stddev:tensor([0.])), value:tensor([0.]): log_prob:tensor(nan)) 


 Address 2 : __A26 
 Value : Variable(name:None, control:True, replace:False, observable:False, observed:False, tagged:False, address:__A26__17, distribution:Normal(mean:tensor([1.]), stddev:tensor([0.1400])), value:tensor([1.1673]): log_prob:tensor(0.3334)) 


 Address 3 : __A47 
 Value : Variable(name:None, control:True, replace:False, observable:False, observed:False, tagged:False, address:__A47__24124, distribution:Uniform(low: tensor([0.]), high: tensor([1.])), value:tensor([0.6805]): log_prob:tensor(0.)) 


 Address 4 : __A27 
 Value : Variable(name:None, control:True, replace:False, observable:False, observed:False, tagged:False, address:__A27__426, distribution:Normal(mean:tensor([0.]), stddev:tensor([0.])), value:tensor([0.]): log

## Generating to the posterior by conditioning on the number of infected individuals for each demographic

In [9]:
 posterior_dist = model.posterior_traces(num_traces=10, inference_engine=pyprob.InferenceEngine.RANDOM_WALK_METROPOLIS_HASTINGS, observe={'n_infect': parse_n_infect_from_output(PATH+'output.txt')})

Removing output...
Time spent  | Time remain.| Progress             | Trace | Accepted|Smp reuse| Traces/sec
Removing output...
Removing output...0:31:20 | ####---------------- |  2/10 |   0.00% | 100.00% | 0.00       
Removing output...0:37:29 | ######-------------- |  3/10 |   0.00% | 100.00% | 0.00       
Removing output...0:36:54 | ########------------ |  4/10 |   0.00% | 100.00% | 0.00       
Removing output...0:33:54 | ##########---------- |  5/10 |   0.00% | 100.00% | 0.00       
Removing output...0:29:29 | ############-------- |  6/10 |   0.00% | 100.00% | 0.00       
Removing output...0:24:13 | ##############------ |  7/10 |   0.00% | 100.00% | 0.00       
Removing output...0:18:34 | ################---- |  8/10 |   0.00% | 100.00% | 0.00       
Removing output...0:12:34 | ##################-- |  9/10 |   0.00% | 100.00% | 0.00       
Removing output...0:06:22 | #################### | 10/10 |   0.00% | 100.00% | 0.00       



In [15]:
print(posterior_dist.values_numpy())

[Trace(all:492,230, controlled:492,229, replaced:0, observeable:1, observed:1, tagged:0, uncontrolled:0, log_prob:tensor(nan), log_importance_weight:0.0)
 Trace(all:492,230, controlled:492,229, replaced:0, observeable:1, observed:1, tagged:0, uncontrolled:0, log_prob:tensor(nan), log_importance_weight:0.0)
 Trace(all:492,230, controlled:492,229, replaced:0, observeable:1, observed:1, tagged:0, uncontrolled:0, log_prob:tensor(nan), log_importance_weight:0.0)
 Trace(all:492,230, controlled:492,229, replaced:0, observeable:1, observed:1, tagged:0, uncontrolled:0, log_prob:tensor(nan), log_importance_weight:0.0)
 Trace(all:492,230, controlled:492,229, replaced:0, observeable:1, observed:1, tagged:0, uncontrolled:0, log_prob:tensor(nan), log_importance_weight:0.0)
 Trace(all:492,230, controlled:492,229, replaced:0, observeable:1, observed:1, tagged:0, uncontrolled:0, log_prob:tensor(nan), log_importance_weight:0.0)
 Trace(all:492,230, controlled:492,229, replaced:0, observeable:1, observed:

### Inference with Importance sampling

In [16]:
posterior_dist_IS = model.posterior_traces(num_traces=10, inference_engine=pyprob.InferenceEngine.IMPORTANCE_SAMPLING, observe={'n_infect': parse_n_infect_from_output(PATH+'output.txt')})

Time spent  | Time remain.| Progress             | Trace | Traces/sec
Removing output...
Removing output...0:33:27 | ####---------------- |  2/10 | 0.00       
Removing output...0:39:10 | ######-------------- |  3/10 | 0.00       
Removing output...0:38:51 | ########------------ |  4/10 | 0.00       
Removing output...0:35:51 | ##########---------- |  5/10 | 0.00       
Removing output...0:31:04 | ############-------- |  6/10 | 0.00       
Removing output...0:25:32 | ##############------ |  7/10 | 0.00       
Removing output...0:19:37 | ################---- |  8/10 | 0.00       
Removing output...0:13:20 | ##################-- |  9/10 | 0.00       
Removing output...0:06:45 | #################### | 10/10 | 0.00       



#### View observations

In [3]:
# ground_truth_trace = next(model._trace_generator())
prior.named_variables['n_infect'].value

AttributeError: 'Empirical' object has no attribute 'named_variables'

In [ ]:
is_posterior_traces = model.posterior_traces(observe={'n_infect': ground_truth_prevalence}, num_traces=5)
is_posterior_population = is_posterior_traces.map(lambda trace: trace.named_variables['population'].value)
is_posterior_prevalence = is_posterior_population.map(get_prevalence)

## Plot Histogram of Addresses

In [ ]:
# we currently do not have a ground truth for this scenario run with a small pop size
pyprob.diagnostics.address_histograms(dists, plot=True, plot_show=True, file_name=file_name, ground_truth_trace=ground_truth_trace, use_address_base=use_address_base)

 Address 1 : [forward()+0x1f3; OM::Simulator::start(scnXml::Monitoring const&)+0x28a; OM::Population::createInitialHumans()+0x94; OM::Population::newHuman(OM::SimTime)+0x5c; OM::Host::Human::Human(OM::SimTime)+0x12b; OM::WithinHost::WHInterface::createWithinHostModel(double)+0x72; OM::WithinHost::CommonWithinHost::CommonWithinHost(double)+0x3a; OM::WithinHost::WHFalciparum::WHFalciparum(double)+0xe6; OM::util::random::gauss(double, double)+0xb4]__Normal 
 Value : Variable(name:None, control:True, replace:False, observable:False, observed:False, tagged:False, address:[forward()+0x1f3; OM::Simulator::start(scnXml::Monitoring const&)+0x28a; OM::Population::createInitialHumans()+0x94; OM::Population::newHuman(OM::SimTime)+0x5c; OM::Host::Human::Human(OM::SimTime)+0x12b; OM::WithinHost::WHInterface::createWithinHostModel(double)+0x72; OM::WithinHost::CommonWithinHost::CommonWithinHost(double)+0x3a; OM::WithinHost::WHFalciparum::WHFalciparum(double)+0xe6; OM::util::random::gauss(double, doub